In [1]:
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')

from utils_libs import *
from utils_data_prep import *

from scipy.stats import lognorm
from scipy.stats import norm
# from scipy.stats import chisqprob

from numpy import prod
import seaborn as sns

# statiscal models
import statsmodels as sm
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.api import VAR, DynamicVAR

from statsmodels.stats import diagnostic
import datetime
import pickle
from datetime import timedelta
import math

SyntaxError: Missing parentheses in call to 'print' (utils_data_prep.py, line 117)

In [2]:
#import transaction data market 1 & 2

trx_df_market1 = np.load("../../dataset/bitcoin/large/trx_df_tick_data_bitfinex_market.dat")
print(trx_df_market1.shape)
print(trx_df_market1.head(3))

trx_df_market2 = np.load("../../dataset/bitcoin/large/trx_df_tick_data_bitstamp_market.dat")
print(trx_df_market2.shape)
print(trx_df_market2.head(3))

print('TRX data LOADED...\n')

(5284899, 6)
    timestamp   price       cost  amount  side        date_time
0  1527803735  7555.4   506.2118   0.067  sell  2018-5-31 21-55
1  1527803738  7555.4  6044.3200   0.800  sell  2018-5-31 21-55
2  1527803738  7555.4    75.5540   0.010  sell  2018-5-31 21-55
(2698796, 6)
    timestamp    price         cost    amount  side       date_time
0  1527800413  7538.34    75.383400  0.010000  sell  2018-5-31 21-0
1  1527800418  7533.92    35.761333  0.004747  sell  2018-5-31 21-0
2  1527800418  7535.44  2995.337400  0.397500  sell  2018-5-31 21-0
TRX data LOADED...



In [3]:
# extract transaction features fast

a = datetime.datetime.now()
trx_features_market1, trx_features_mins_market1 = extract_trx_features_matrix_fast(trx_df_market1)
b = datetime.datetime.now()
print('Finished extracting TRX features for market1')
print(b-a)
print(trx_features_market1.shape)
print(trx_features_market1[1:5, 0:7])     
print(trx_features_mins_market1[0:5])


a = datetime.datetime.now()
trx_features_market2, trx_features_mins_market2 = extract_trx_features_matrix_fast(trx_df_market2)
b = datetime.datetime.now()
print('Finished extracting TRX features for market 2')
print(b-a)
print(trx_features_market2.shape)
print(trx_features_market2[1:5, 0:7])   
print(trx_features_mins_market2[0:5])

Finished extracting TRX features for market1
0:00:15.143292
(171727, 7)
[[ 10.69301246   0.27047247  10.42253999 -10.15206752  24.
   13.          11.        ]
 [  3.47818415   0.37418415   3.104       -2.72981585  14.
    8.           6.        ]
 [  3.41684073   0.72038123   2.6964595   -1.97607827  14.
    6.           8.        ]
 [  1.70283041   0.244        1.45883041  -1.21483041  11.
    5.           6.        ]]
['2018-5-31 21-55', '2018-5-31 21-56', '2018-5-31 21-57', '2018-5-31 21-58', '2018-5-31 21-59']
Finished extracting TRX features for market 2
0:00:09.518205
(168743, 7)
[[ 1.16587643  0.21889568  0.94698075 -0.72808507 11.          5.
   6.        ]
 [ 0.09794554  0.01652677  0.08141877 -0.064892    4.          2.
   2.        ]
 [ 5.58204274  4.21884088  1.36320186  2.85563902 10.          5.
   5.        ]
 [ 2.11173496  0.05254428  2.05919068 -2.0066464  12.          4.
   8.        ]]
['2018-5-31 21-0', '2018-5-31 21-1', '2018-5-31 21-10', '2018-5-31 21-11', '2018-

In [6]:
# import OB data for market 1

all_dta_minu_ob_market1 = np.load("../../dataset/bitcoin/large/dta_minu_bitfinex_ob.dat")
all_loc_hour_ob_market1 = np.load("../../dataset/bitcoin/large/loc_hour_bitfinex_ob.dat")
all_loc_month_ob_market1 = np.load("../../dataset/bitcoin/large/loc_month_bitfinex_ob.dat")
all_dta_minu_ticks_ob_market1 = np.load("../../dataset/bitcoin/large/dta_minu_ticks_bitfinex_ob.dat")
print('OB data LOADED...\n')

print len(all_dta_minu_ob_market1), len(all_loc_hour_ob_market1), len(all_loc_month_ob_market1)

# How data looks and is ogranised
print( np.shape(all_dta_minu_ob_market1) )
min_idx = 5; #minute offset 
bid_idx = 1; #bid or ask side
print( 'price amount: \n' )
print( all_dta_minu_ob_market1[min_idx][bid_idx] )

In [4]:
# import OB features for market 1

ob_features_minu_market1 = np.load("../../dataset/bitcoin/large/feature_minu_bitfinex_ob.dat")
ob_features_minu_ticks_market1 = np.load("../../dataset/bitcoin/large/feature_minu_ticks_bitfinex_ob.dat")

print('OB features LOADED...\n')

print(ob_features_minu_market1.shape)
print(ob_features_minu_market1[1:5,0:7])  
print(ob_features_minu_ticks_market1[1:5])
print(ob_features_minu_ticks_market1.shape)

OB features LOADED...

(174410, 10)
[[1.00000000e-01 6.98505000e+01 2.00000000e+03 2.00000000e+03
  0.00000000e+00 5.99030876e+03 5.78072981e+03]
 [1.00000000e-01 6.80830000e+01 2.00000000e+03 2.00000000e+03
  0.00000000e+00 5.96463486e+03 5.75689999e+03]
 [1.00000000e-01 6.89980000e+01 2.00000000e+03 2.00000000e+03
  0.00000000e+00 5.99457686e+03 5.79781907e+03]
 [1.00000000e-01 7.20640000e+01 2.00000000e+03 2.00000000e+03
  0.00000000e+00 6.06418615e+03 5.82049270e+03]]
['2018-5-31 22-1' '2018-5-31 22-2' '2018-5-31 22-3' '2018-5-31 22-4']
(174410,)


In [5]:
# prepare data for learning (yi, ti) = yi is target volume, ti is the key in the feature_dict
# sources = 1..M., feature_dist[key=(ti, source_id)]=matrix of features

from datetime import datetime
import time

def get_source_feature_matrix(target_idx, features_tuple, feature_dict, source_id, delta1, delta2):
    # y_i target volume extracted from index at target_idx
    # features_tuple - (trx_features_timestamp_market1_sort, trx_features_market1_sort) 
    # feature_dict , #key = (target_id, source_id), value = matrix of features from source_id; - call by reference
    # history [t_i-delta2,t_i-delta1], where t_i is time of target volume y_i
    
    trx_features_timestamp_market_sort = features_tuple[0]
    trx_features_market_sort = features_tuple[1]

    delta = delta2 - delta1;
    ti = trx_features_timestamp_market_sort[target_idx]
    tj = trx_features_timestamp_market_sort[target_idx - delta2]

    yi = trx_features_market_sort[target_idx, 0]
    #(yi,ti) target_var, feature_dict[(ti,source_id)] = features

    if (ti - tj == (delta*60)):
        
        tmp_features_matrix = trx_features_market_sort[target_idx-delta2+1:target_idx-delta1+1,:]
        key_tmp = (ti, source_id)
        feature_dict[key_tmp]=tmp_features_matrix
        
    else:
        
        tmp_features_matrix = np.zeros([delta, trx_features_market_sort.shape[1]])
        offset_idx = 1
        while( (target_idx>=offset_idx) & ((ti-trx_features_timestamp_market_sort[target_idx-offset_idx]) <= (delta*60)) ):
            tmp_feature_offset_idx = delta-((ti-trx_features_timestamp_market_sort[target_idx-offset_idx])//60)
            tmp_features_matrix[tmp_feature_offset_idx,:] = trx_features_market_sort[target_idx-offset_idx,:]    
            offset_idx += 1

        key_tmp = (ti, source_id)
        feature_dict[key_tmp]=tmp_features_matrix
        
    return (yi,ti)

def fillEmptyFeat(tmp_key, my_dict, elem):
    if tmp_key not in my_dict:
        my_dict[tmp_key] = elem
        return 1
    else:
        return 0

def extract_target_feature_pairs(source_data, feature_dict, delta1, delta2):
    
    source_type = source_data[0]
    source_id = source_data[1]
    print(source_type)
    print("source id = " + str(source_id))
    
    
    features_mins_market = source_data[2]
    features_market = source_data[3]

    features_timestamp_market=[int((datetime.strptime(str_var, '%Y-%m-%d %H-%M')-datetime(1970,1,1)).total_seconds())
                                    for str_var in features_mins_market]
    idx_sort_time = np.argsort(features_timestamp_market)

    features_timestamp_market_sort = [features_timestamp_market[x] for x in idx_sort_time]
    features_market_sort = features_market[idx_sort_time,:]

    features_time_tuple_sort = (features_timestamp_market_sort, features_market_sort)

    target_feature_key = [get_source_feature_matrix(idx, 
                                                    features_time_tuple_sort, 
                                                    feature_dict, 
                                                    source_id, 
                                                    delta1, 
                                                    delta2) 
                          for idx in range(len(features_timestamp_market_sort))]
   
    
    print("Extracted!")
    return target_feature_key
    
    
    
feature_dict={}
#key = (timestep, source_id), value = matrix 

delta2 = 11 # look-back time steps
delta1 = 1 # k-step ahead 
delta = delta2 - delta1


source1_trx = ("trx-data", 1, trx_features_mins_market1, trx_features_market1)
# target 
target_feature_key = extract_target_feature_pairs(source1_trx, feature_dict, delta1, delta2)


source2_trx = ("trx-data", 2, trx_features_mins_market2, trx_features_market2)
#for second market we neglect target_feature_key, it only inserts features for key with source2
_ = extract_target_feature_pairs(source2_trx, feature_dict, delta1, delta2)


source3_ob = ("ob-data", 3, ob_features_minu_ticks_market1, ob_features_minu_market1)
_ = extract_target_feature_pairs(source3_ob, feature_dict, delta1, delta2)

source3_ob = ("ob-data", 3, ob_features_minu_ticks_market1, ob_features_minu_market1)
_ = extract_target_feature_pairs(source3_ob, feature_dict, delta1, delta2)


empty_elem = np.zeros([delta, trx_features_market2.shape[1] ])
tmp_inserts = map( lambda x: fillEmptyFeat((x[1],2), feature_dict, empty_elem), target_feature_key)
    
empty_elem = np.zeros([delta, ob_features_minu_market1.shape[1] ])
tmp_inserts = map( lambda x: fillEmptyFeat((x[1],3), feature_dict, empty_elem), target_feature_key)

print("finished")


trx-data
source id = 1
Extracted!
trx-data
source id = 2
Extracted!
ob-data
source id = 3
Extracted!
finished


In [2]:
import pickle

path_data = "../../dataset/bitcoin/"

dta = pickle.load(open(path_data + 'target_market1_features_ob_trx_delta_5_data.pkl', "rb"), encoding = 'latin1')

print(np.shape(dta[0][2][2]))

(9, 13)


In [6]:
# ----- create target, time, feature tupple (yi, ti, xi)

# target_feature_key [y_i, timestep]

# feature_dict [(timestep, source_id), matrix]

# source1 feature matrix, source2 feature matrix, source 3 feature matrix


target_feature_data = [(x[0], x[1], [feature_dict[(x[1],1)], feature_dict[(x[1],2)], feature_dict[(x[1],3)]]) 
                       for x in target_feature_key]

print(len(target_feature_data))
# print(target_feature_data[10])

171727


In [34]:
# first column corresponds to the target variable
# timestamp increasing order 
target_feature_data[8]

(0.9585924499999999,
 1527804180,
 [array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00],
         [ 5.69051264e+01,  3.63463160e-01,  5.65416632e+01,
          -5.61782000e+01,  3.50000000e+01,  6.00000000e+00,
           2.90000000e+01],
         [ 1.06930125e+01,  2.70472470e-01,  1.04225400e+01,
          -1.01520675e+01,  2.40000000e+01,  1.30000000e+01,
           1.10000000e+01],
         [ 3.47818415e+00,  3.74184150e-01,  3.10400000e+00,
          -2.72981585e+00,  1.40000000e+01,  8.00000000e+00,
           6.00000000e+00],
         [ 3.41684073e+00,  7.20381230e-01,  2.69645950e+00,
          -1.97607827e+00,  1.40000000e+01,  6.00000000e+00,
           8.00000000e+00],
         [ 1.70283041e+00,  2.44000000e-01,  1.45883041e+00,
     

In [10]:
# load organized data

import pickle

filename = '../../dataset/bitcoin/delta_15_data.pkl'

# filename = '../../dataset/bitcoin/delta_15_data.pkl'

infile = open(filename,'rb')
target_feature_data_agg = pickle.load(infile)
infile.close()
print("loaded")

print(len(target_feature_data_agg))

print(target_feature_data_agg[1])

print([i[1] for i in target_feature_data_agg[0:5]])

loaded
11449
(552.8473395399999, 1527804600, (array([[ 5.16000000e-02,  4.10000000e-02,  1.06000000e-02,
         3.04000000e-02,  3.00000000e+00,  1.00000000e+00,
         2.00000000e+00],
       [ 1.79610000e-01,  1.69000000e-01,  1.06100000e-02,
         1.58390000e-01,  5.00000000e+00,  4.00000000e+00,
         1.00000000e+00],
       [ 9.58592450e-01,  9.04882450e-01,  5.37100000e-02,
         8.51172450e-01,  1.10000000e+01,  8.00000000e+00,
         3.00000000e+00],
       [ 2.22907820e-01,  2.22907820e-01,  0.00000000e+00,
         2.22907820e-01,  4.00000000e+00,  4.00000000e+00,
         0.00000000e+00],
       [ 2.19642433e+02,  8.12402879e+01,  1.38402145e+02,
        -5.71618573e+01,  2.60000000e+02,  1.29000000e+02,
         1.31000000e+02],
       [ 8.22609354e+01,  4.27701272e+01,  3.94908082e+01,
         3.27931904e+00,  1.40000000e+02,  6.80000000e+01,
         7.20000000e+01],
       [ 3.96537351e+01,  1.96408320e+01,  2.00129031e+01,
        -3.72071070e-01,  1.560

In [ ]:
target_feature_data = target_feature_data_agg


In [11]:
# remove correlated features

target_feature_data = []

for i in target_feature_data_agg:
    
    tmp_ob = i[2][2]
    
    ob_feature = tmp_ob[:, [0, 1, 3, 5, 6, 8, 9]]
    
    target_feature_data.append([i[0], i[1], [i[2][0], i[2][1], ob_feature]])
    
print(target_feature_data[1])

[552.8473395399999, 1527804600, [array([[ 5.16000000e-02,  4.10000000e-02,  1.06000000e-02,
         3.04000000e-02,  3.00000000e+00,  1.00000000e+00,
         2.00000000e+00],
       [ 1.79610000e-01,  1.69000000e-01,  1.06100000e-02,
         1.58390000e-01,  5.00000000e+00,  4.00000000e+00,
         1.00000000e+00],
       [ 9.58592450e-01,  9.04882450e-01,  5.37100000e-02,
         8.51172450e-01,  1.10000000e+01,  8.00000000e+00,
         3.00000000e+00],
       [ 2.22907820e-01,  2.22907820e-01,  0.00000000e+00,
         2.22907820e-01,  4.00000000e+00,  4.00000000e+00,
         0.00000000e+00],
       [ 2.19642433e+02,  8.12402879e+01,  1.38402145e+02,
        -5.71618573e+01,  2.60000000e+02,  1.29000000e+02,
         1.31000000e+02],
       [ 8.22609354e+01,  4.27701272e+01,  3.94908082e+01,
         3.27931904e+00,  1.40000000e+02,  6.80000000e+01,
         7.20000000e+01],
       [ 3.96537351e+01,  1.96408320e+01,  2.00129031e+01,
        -3.72071070e-01,  1.56000000e+02,  6

# --- Load organized data

In [35]:
import pickle

path_data = "../../dataset/bitcoin/"

dta = pickle.load(open(path_data + 'target_market1_features_ob_trx_delta_1_data.pkl', "rb"), encoding = 'latin1')

print(np.shape(dta[0][2][2]))


target_feature_data = dta

(9, 13)


In [36]:
# ----- dataset split

cnt = len(target_feature_data)

train_split = int(cnt*0.7)
val_split = int(cnt*0.8)

train_data = target_feature_data[:train_split]
val_data = target_feature_data[train_split:val_split]
test_data = target_feature_data[val_split:]

print(len(train_data), len(val_data), len(test_data))

120208 17173 34346


In [32]:
# ----- normalization

# training data: mean and standard deviation from the training data
src_x = []
src_num = len(train_data[0][2])

src_m = []
src_std = []

for src_idx in range(src_num):
    
    # [N T D]     
    tmp_src = [tmp[2][src_idx] for tmp in train_data]
    
    print('test', np.shape(tmp_src))
    
    tmp_mean = np.mean(tmp_src, axis = 0)
    tmp_std = np.std(tmp_src, axis = 0)
    
    src_m.append(tmp_mean)
    src_std.append(tmp_std)


print((train_data[0][2][0]))
print(val_data[0][2][0])
print(test_data[0][2][0])

# normalize training data
for src_idx in range(src_num):
    for tmp_idx, tmp in enumerate(train_data):
        
        tmpdata = list(tmp)
        
        tmpdata[2] = list(tmpdata[2])
        
        tmpdata[2][src_idx] = (tmpdata[2][src_idx] - src_m[src_idx])/(src_std[src_idx] + 1e-5)
        
        train_data[tmp_idx] = tmpdata
        
    
# normalize validattion data
for src_idx in range(src_num):
    for tmp_idx, tmp in enumerate(val_data):
        
        tmpdata = list(tmp)
        
        tmpdata[2] = list(tmpdata[2])
        
        tmpdata[2][src_idx] = (tmpdata[2][src_idx] - src_m[src_idx])/(src_std[src_idx] + 1e-5)
        
        val_data[tmp_idx] = tmpdata
        
        
# normalize testing data
for src_idx in range(src_num):
    for tmp_idx, tmp in enumerate(test_data):
        
        tmpdata = list(tmp)
        
        tmpdata[2] = list(tmpdata[2])
        
        tmpdata[2][src_idx] = (tmpdata[2][src_idx] - src_m[src_idx])/(src_std[src_idx] + 1e-5)
        
        test_data[tmp_idx] = tmpdata

print("\n")
print(train_data[0][2][0])
print(val_data[0][2][0])
print(test_data[0][2][0])

test (24042, 9, 7)
test (24042, 9, 7)
test (24042, 9, 13)
test (24042, 9, 13)
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[ 7.47109680e-01  2.88002000e-03  7.44229660e-01 -7.41349640e-01
   1.00000000e+01  1.00000000e+00  9.00000000e+00]
 [ 3.84014110e-01  0.00000000e+00  3.84014110e-01 -3.84014110e-01
   5.00000000e+00  0.00000000e+00  5.00000000e+00]
 [ 1.21329256e+00  3.00000000e-02  1.18329256e+00 -1.15329256e+00
   4.00000000e+00  1.00000000e+00  3.00000000e+00]
 [ 1.05275692e+01  4.30000000e+00  6.22756918e+00 -1.92756918e+00
   1.50000000e+01  6.00000000e+00  9.00000000e+00]
 [ 2.51951624e+00  5.00000000e-03  2.51451624e+00 -2.50951624e+00
   6.00000000e+00  1.00000000e+00  5.00000000e+00]
 [ 1.23590140e-01  5.36551000e-02  6.99350400e-02 -1.62799400e-02
   8.00000000e+00  3.00000000e+00  5.00000000e+00]
 [ 

In [27]:
src_m

[array([[13.81086405,  6.47579552,  7.33506853, -0.85927301, 32.31424174,
         15.22577157, 17.08847018],
        [14.17094606,  6.66241658,  7.50852948, -0.84611289, 32.59371101,
         15.32439065, 17.26932036],
        [13.7592665 ,  6.57883558,  7.18043092, -0.60159535, 32.01680393,
         15.04475501, 16.97204891],
        [13.85062336,  6.58659997,  7.26402339, -0.67742341, 32.31049829,
         15.26158389, 17.0489144 ],
        [13.80457962,  6.6467998 ,  7.15777982, -0.51098003, 32.1317694 ,
         15.10298644, 17.02878296],
        [13.78079416,  6.45907475,  7.32171941, -0.86264466, 32.28712254,
         15.19407703, 17.0930455 ],
        [14.19692374,  6.6974328 ,  7.49949094, -0.80205814, 32.62898261,
         15.37218201, 17.2568006 ],
        [13.80800298,  6.59108925,  7.21691374, -0.62582449, 32.10535729,
         15.10128109, 17.0040762 ],
        [13.86544482,  6.58785185,  7.27759297, -0.68974111, 32.35583562,
         15.26416272, 17.09167291]]),
 array([

In [28]:
src_std

[array([[27.76275399, 16.22984909, 19.7157813 , 23.09695993, 39.31264881,
         22.52433129, 24.84174084],
        [31.16712541, 16.55474985, 23.85930759, 26.74439933, 40.17553167,
         22.68038642, 25.54858809],
        [27.67508279, 17.13661669, 18.97273898, 23.2668585 , 38.5910352 ,
         21.95536973, 24.54417084],
        [27.45353068, 16.19263173, 19.496271  , 23.04160287, 39.35574524,
         22.69773304, 24.96640658],
        [27.20906001, 16.92138637, 18.15733714, 22.17456811, 38.77288701,
         22.24196802, 24.75375951],
        [27.71917996, 16.26131474, 19.66647297, 23.10948064, 39.1364436 ,
         22.38939339, 24.80857153],
        [31.17027536, 16.6161435 , 23.84106327, 26.78430472, 40.12624747,
         22.77086059, 25.44044306],
        [27.74772693, 17.08679801, 19.14257113, 23.38497469, 38.75618143,
         22.08225352, 24.59652927],
        [27.48794953, 16.18444038, 19.49623136, 22.9889305 , 39.42788532,
         22.69159091, 25.00106973]]),
 array([

In [33]:
# normalize training data

auto_tr_data = [ [i[0], i[1], [i[2][0], i[2][2]]]  for i in train_data ]
print(np.shape(auto_tr_data[0][2][0]))

auto_val_data = [ [i[0], i[1], [i[2][0], i[2][2]]]  for i in val_data ]
auto_test_data = [ [i[0], i[1], [i[2][0], i[2][2]]]  for i in test_data ]


train_data = auto_tr_data
val_data = auto_val_data
test_data = auto_test_data


(9, 7)


In [34]:
import pickle

pickle.dump(train_data, open("../../dataset/bitcoin/market1_tar5_len10_auto/train.p", "wb" ))
pickle.dump(val_data, open("../../dataset/bitcoin/market1_tar5_len10_auto/val.p", "wb" ))
pickle.dump(test_data, open("../../dataset/bitcoin/market1_tar5_len10_auto/test.p", "wb" ))